In [1]:
import pandas as pd
import numpy as np

Thanks to Kriti Mahajan. This table was developed with study type features extracted from the meta data.
Code found here: https://colab.research.google.com/drive/1lay-yWJ5ohN94lQLHYIxtWMuDWtCRGy0#scrollTo=N-IhAeEkcxDe

Strategy:
1. Combine newest CoY dataset, Kaggle dataset, PubMed dataset and other annotation group's dataset all into one annotation table.
2. Feature engineer via revising ngram list/reverse engineer
3. If desperate, get synthetic labels via synthetic augmentation of text data (trade out word for synonym).

### Trying direct merge with mf and kaggle label set via sha.

In [151]:
mf = pd.read_csv('metadata_features-2.csv')
mf

,Unnamed: 0,paper_id,sha,title,abstract,publish_time,authors,url,Statistical Methods,statistical_method_details,Study Type,study_type_details,results,coronavirus,fatality,sample_size,paper_focus,variables_controlled_for
0,0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,Sequence requirements for RNA strand transfer ...,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",http://europepmc.org/articles/pmc125340?pdf=re...,[],NaN,['mutagenesis'],Here we present results of a comprehensive co-...,Here we present results of a comprehensive co-...,NaN,NaN,NaN,NaN,NaN
1,1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,"Crystal structure of murine sCEACAM1a[1,4]: a ...",CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",http://europepmc.org/articles/pmc125375?pdf=re...,[],NaN,[],NaN,Here we report the crystal structure of solubl...,NaN,NaN,NaN,NaN,NaN
2,2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,Synthesis of a novel hepatitis C virus protein...,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,[],NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,Structure of coronavirus main proteinase revea...,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",http://europepmc.org/articles/pmc126080?pdf=re...,['model'],Molecular modelling and mutagenesis data impli...,['mutagenesis'],Molecular modelling and mutagenesis data impli...,The study illustrates that RNA viruses have ev...,NaN,NaN,NaN,NaN,NaN
4,4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,Discontinuous and non-discontinuous subgenomic...,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M...",http://europepmc.org/articles/pmc136939?pdf=re...,[],NaN,[],NaN,We show that BEV mRNAs 3–5 lack a leader seque...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59125,59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,Cutting-Edge Infectious Disease Diagnostics wi...,"Three recent Science articles (Chen et al., 20...",2018-06-13,"Chiu, Charles",https://doi.org/10.1016/j.chom.2018.05.016,[],NaN,['clinical'],"Three recent Science articles (Chen et al., 20...",NaN,NaN,NaN,NaN,NaN,NaN
59126,59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,Intranasal administration of oxytocin: Behavio...,Abstract The intranasal (IN-) administration o...,2013-09-30,"Veening, Jan G.; Olivier, Berend",https://doi.org/10.1016/j.neubiorev.2013.04.012,[],NaN,['review'],The present review explores our current knowle...,Only very recently in a microdialysis study in...,NaN,NaN,NaN,NaN,NaN
59127,59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,Infecciones virales graves en pacientes inmuno...,Resumen Los virus tienen un papel importante d...,2011-04-30,"Díaz, A.; Zaragoza, R.; Granada, R.; Salavert, M.",https://doi.org/10.1016/j.medin.2011.01.001,[],NaN,['review'],A review of the literature has been performed ...,NaN,NaN,"Furthermore, ventilator-associated pneumonia (...",NaN,NaN,NaN
59128,59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,"18 Omics, Bioinformatics, and Infectious Disea...",Bioinformatics and the ‘omics’ studies are rap...,2011-12-31,"Paszkiewicz, Konrad H.; Giezen, Mark van der",https://doi.org/10.1016/b978-0-12-384890-1.000...,"['sequencing', 'methods']","In addition, we provide an overview of methods...",['bioinformatics'],Bioinformatics and the ‘omics’ studies are rap...,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
list(mf)

['Unnamed: 0',
 'paper_id',
 'sha',
 'title',
 'abstract',
 'publish_time',
 'authors',
 'url',
 'Statistical Methods',
 'statistical_method_details',
 'Study Type',
 'study_type_details',
 'results',
 'coronavirus',
 'fatality',
 'sample_size',
 'paper_focus',
 'variables_controlled_for']

In [153]:
features = mf.drop(columns = ['Unnamed: 0',
 'title',
 'abstract',
 'publish_time',
 'authors',
 'url',
 'Statistical Methods',
 'statistical_method_details',
 'study_type_details',
 'results',
 'coronavirus',
 'fatality',
 'sample_size',
 'paper_focus',
 'variables_controlled_for'])
features

,paper_id,sha,Study Type
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis']
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,[]
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,[]
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis']
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,[]
...,...,...,...
59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical']
59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review']
59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review']
59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics']


In [154]:
features['Study Type'].value_counts()

[]                                                                           37266
['clinical']                                                                  5782
['review']                                                                    3594
['genetic']                                                                   3089
['review', 'clinical']                                                        1347
                                                                             ...  
['retrospective study', 'retrospectively analyzed']                              1
['retrospective study', 'histories']                                             1
['review', 'retrospective study', 'retrospectively analyzed', 'clinical']        1
['review', 'single center study', 'clinical']                                    1
['meta-analysis', 'clinical', 'bioinformatics']                                  1
Name: Study Type, Length: 393, dtype: int64

In [155]:
to_drop = ['[]']
features = features[~features['Study Type'].isin(to_drop)]
features

,paper_id,sha,Study Type
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis']
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis']
6,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']"
7,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic']
10,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic']
...,...,...,...
59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical']
59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review']
59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review']
59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics']


In [156]:
features['Study Type'].value_counts()

['clinical']                                                        5782
['review']                                                          3594
['genetic']                                                         3089
['review', 'clinical']                                              1347
['experimental']                                                    1241
                                                                    ... 
['review', 'retrospective cohort', 'case series', 'clinical']          1
['review', 'retrospective study', 'clinical', 'laboratory data']       1
['experimental', 'meta-analysis']                                      1
['time series', 'time-series', 'clinical']                             1
['mutagenesis', 'review', 'life cycle']                                1
Name: Study Type, Length: 392, dtype: int64

In [157]:
kaggle_labels = pd.read_csv('design-2.csv')

In [181]:
kg_merge = features.merge(kaggle_labels, how='inner', left_on='sha', right_on='id')
kg_merge

,paper_id,sha,Study Type,label,id
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e
...,...,...,...,...,...
493,dycx8i7h,2d0d272a793b99ef896547024c51f486e9552e5a,['genetic'],0,2d0d272a793b99ef896547024c51f486e9552e5a
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b
496,i2vg8zqc,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802,"['experimental', 'clinical']",0,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802


Complete kg_merge annotation set by:
1. mapping labels to cw_labels
2. performing one-hot-encoding to output multi-label dataset

In [329]:
cw_labels = pd.read_csv('CoY_newlist.csv')

In [330]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional.1',
 'Clinical-observational',
 'Clinical-observational.1',
 'Systematic review and/or meta-analysis.1',
 'Other',
 'General Notes']

In [331]:
cw_labels = cw_labels.drop(columns = ['Other', 'General Notes'])

In [332]:
#Rename/Clean-up labels
cw_labels = cw_labels.rename(columns={'Computational ': 'Computational', 'Experimental-in vitro' : 'Experimental - in vitro', 'Experimental-in vivo' : 'Experimental - in vivo', 'Clinical-interventional.1' : 'Clinical-interventional', 'Clinical-observational.1' : 'Clinical-observational', 'Systematic review and/or meta-analysis ' : 'Systematic review and/or meta-analysis'})

In [333]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional',
 'Clinical-observational',
 'Clinical-observational',
 'Systematic review and/or meta-analysis.1']

Mapping Kaggle labels to CoronaWhy labels

0. Other -> Drop
1. Systematic review -> Systematic review and/or meta-analysis
2. Randomized control trial -> Clinical-interventional
3. Non-randomized trial -> Clinical - interventional
4. Prospective observational -> Clinical -observational
5. Time-to-event analysis -> Clinical-observational
6. Retrospective observational -> Clinical-observational
7. Cross-sectional -> Clinical-observational
8. Case series -> Clinical-observational
9. Modeling -> Computational

In [187]:
kg_merge.explode('label')
kg_merge['label'].unique()

array([1, 8, 5, 7, 4, 2, 9, 6, 0, 3])

In [188]:
dic_map={
         1:'Systematic review and/or meta-analysis',
         2:'Clinical-interventional',
         3:'Clinical-interventional',
         4:'Clinical-observational',
         5:'Clinical-observational',
         6:'Clinical-observational',
         7:'Clinical-observational',
         8:'Clinical-observational',
         9:'Computational',
         0:'Other'
        }

In [189]:
kg_merge['mapped'] = kg_merge.apply(lambda x: dic_map[x.label], axis = 1) 
kg_merge.head()

,paper_id,sha,Study Type,label,id,mapped
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,Systematic review and/or meta-analysis
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,Clinical-observational
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,Clinical-observational
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,Clinical-observational
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,Clinical-observational


In [190]:
#Drop 0. Other label
kg_merge = kg_merge[kg_merge.label != 0]
kg_merge

,paper_id,sha,Study Type,label,id,mapped
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,Systematic review and/or meta-analysis
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,Clinical-observational
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,Clinical-observational
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,Clinical-observational
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,Clinical-observational
...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],2,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,Clinical-interventional
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],9,eb720970f362e4416b73393f230c8606c837c6dc,Computational
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,Clinical-interventional
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b,Clinical-observational


100 less annotations is a huge drop. 

### One-Hot Encoding

In [191]:
one_hot = pd.get_dummies(kg_merge['mapped'])
#drop mapped column as it is encoded now
kg_merge = kg_merge.drop('mapped',axis = 1)
# Join the encoded mapped to df3
kg_merge = kg_merge.join(one_hot)
kg_merge

,paper_id,sha,Study Type,label,id,Clinical-interventional,Clinical-observational,Computational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,0,0,0,1
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,0,1,0,0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,0,1,0,0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,0,1,0,0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,0,1,0,0
...,...,...,...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],2,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,1,0,0,0
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],9,eb720970f362e4416b73393f230c8606c837c6dc,0,0,1,0
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,1,0,0,0
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b,0,1,0,0


### Clean up table by dropping unnecessary columns and adding necessary columns to kg_merge to final output.

In [192]:
kg_merge = kg_merge.drop(columns = ['label', 'id'])
kg_merge

,paper_id,sha,Study Type,Clinical-interventional,Clinical-observational,Computational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],0,0,0,1
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",0,1,0,0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",0,1,0,0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],0,1,0,0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",0,1,0,0
...,...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],1,0,0,0
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],0,0,1,0
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],1,0,0,0
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",0,1,0,0


In [193]:
#Adding necessary columns
c = {'Experimental - in vitro': 0, 'Experimental - in vivo': 0}
kg_merge = kg_merge.assign(**c)
#Reordering columns
cols = ['paper_id', 'sha', 'Study Type', 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']
kg_merge = kg_merge[cols]
kg_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],0,0,0,0,0,1
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",0,0,0,0,1,0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",0,0,0,0,1,0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],0,0,0,0,1,0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],0,0,0,1,0,0
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],1,0,0,0,0,0
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],0,0,0,1,0,0
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",0,0,0,0,1,0


In [315]:
kg_merge.iloc[:,3:].sum()
# total1 = []
# for i in range(0,6):
#     total1.append(kg_merge.iloc[:,3:].sum()[i])
# sum(total1) 

Computational                              37
Experimental - in vitro                     0
Experimental - in vivo                      0
Clinical-interventional                    44
Clinical-observational                    252
Systematic review and/or meta-analysis     65
dtype: int64

### Onto cw_labels. 

1. Merge with features.
2. Combine like columns into 1 column.
2. Drop unncessary columns.
3. Reorder columns as in kg_merge.

In [334]:
cw_labels.iloc[:,4:]

,url,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis.1
0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://doi.org/10.1016/j.vaccine.2016.04.096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://doi.org/10.1016/j.jff.2013.04.018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://doi.org/10.1016/j.cell.2006.02.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,https://doi.org/10.1016/j.enbuild.2017.08.025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,https://doi.org/10.1016/j.virol.2012.11.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [335]:
cw_labels.iloc[:,5:].sum()
total3 = []
for i in range(0,6):
    total3.append(cw_labels.iloc[:,5:].sum()[i])
sum(total3) 

412.0

412 total annotations. This does not mean that 427 publications are annotated since each pubication can have more than one label. The number of labels can be reduced based on how many matches there are between cw_labels and features.

In [340]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional',
 'Clinical-observational',
 'Clinical-observational',
 'Systematic review and/or meta-analysis.1']

In [373]:
#Drop empty annotations.
cw_labels.dropna(subset=['Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional',
 'Clinical-observational',
 'Clinical-observational',
 'Systematic review and/or meta-analysis.1'], how='all', inplace = True) 
cw_labels

,Assignee,cord_uid,sha,title,url,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis.1
5,Zayd,y8z0pwgn,8c71f856263c447ba7983e1f7087259008ca8505,A permanent host shift of rabies virus from Ch...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,cxwucrgo,abc25b132b99ecba2d9f81806d2c211037ba1bba,Structural maturation of rubella virus in the ...,https://doi.org/10.1016/s0042-6822(03)00384-2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Lolita,u4mvk89w,7d5cd4faf19ae8838653aa65ea172c97f8bfe9f4,X-Ray Structure and Inhibition of 3C-like Prot...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Lolita,ff9vfrvc,9b07951ffeb24912f35e85556e35108481b02755,Impact of SARS on healthcare utilization by di...,https://doi.org/10.1016/j.healthpol.2007.03.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
14,Lolita,r41ilao6,6bde3798df501fc40dd89e2cff0a1a8fd2b9dd47,Molecular analyses of disease pathogenesis: Ap...,https://doi.org/10.1016/j.vetimm.2005.02.015,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,NaN,a8smmtla,d0146d4c8a05561b617e944b7f7c523c46612dc8,Myricetin: A Dietary Molecule with Diverse Bio...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
845,NaN,s5afikdu,5bf2ab79a25fdb03f50dafb0d08de60acbd50af0,"Cell surface α2,3-linked sialic acid facilitat...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
847,Julie Kurek,n6obzraw,5171ad879a46d32ccb76fcce6c39023aa836db31,2010: The Year of the Lung,https://doi.org/10.1378/chest.10-2555,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848,NaN,z36p5jo6,6aae752380943069e39342f1d86741124211c312,Detection and characterisation of coronaviruse...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1,NaN


577 total full annotations.

In [419]:
cw_merge = features.merge(cw_labels, how='inner', left_on='paper_id', right_on='cord_uid')
cw_merge

,paper_id,sha_x,Study Type,Assignee,cord_uid,sha_y,title,url,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis.1
0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],NaN,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,Protein secretion in Lactococcus lactis : an e...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],NaN,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,A Severe Acute Respiratory Syndrome extranet: ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
2,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],NaN,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,Pneumothorax and mortality in the mechanically...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],NaN,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,Selection of a set of reliable reference genes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,['questionnaire'],NaN,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,"The Trojan Chicken Study, Minnesota",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,"['experimental', 'clinical']",NaN,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,BCX4430 – A broad-spectrum antiviral adenosine...,https://doi.org/10.1016/j.jiph.2016.04.002,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
256,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,['review'],NaN,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,"The Isolation, Pathogenesis, Diagnosis, Transm...",https://doi.org/10.1016/j.cvex.2010.05.014,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1,NaN
257,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,['clinical'],NaN,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,Use of a Multiplex RT-PCR Assay for Simultaneo...,https://doi.org/10.1016/s1671-2927(09)60189-9,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
258,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,['clinical'],GD/ lolita,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,Case control study to investigate risk factors...,https://doi.org/10.1016/j.prevetmed.2012.02.008,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


Only 260 matching publications between cw_labels and features df.

In [427]:
cw_merge.iloc[:,8:]

,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
255,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1,NaN
257,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
258,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [435]:
cw_merge['Clinical-observational'] = cw_merge['Clinical-observational'].astype('float64')

In [436]:
cw_merge

,paper_id,sha_x,Study Type,Assignee,cord_uid,sha_y,title,url,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis
0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],NaN,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,Protein secretion in Lactococcus lactis : an e...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],NaN,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,A Severe Acute Respiratory Syndrome extranet: ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],NaN,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,Pneumothorax and mortality in the mechanically...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],NaN,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,Selection of a set of reliable reference genes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,['questionnaire'],NaN,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,"The Trojan Chicken Study, Minnesota",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,"['experimental', 'clinical']",NaN,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,BCX4430 – A broad-spectrum antiviral adenosine...,https://doi.org/10.1016/j.jiph.2016.04.002,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
256,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,['review'],NaN,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,"The Isolation, Pathogenesis, Diagnosis, Transm...",https://doi.org/10.1016/j.cvex.2010.05.014,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
257,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,['clinical'],NaN,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,Use of a Multiplex RT-PCR Assay for Simultaneo...,https://doi.org/10.1016/s1671-2927(09)60189-9,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
258,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,['clinical'],GD/ lolita,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,Case control study to investigate risk factors...,https://doi.org/10.1016/j.prevetmed.2012.02.008,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [437]:
cw_merge.iloc[:,8:].sum()

Computational                             25.0
Experimental - in vitro                   58.0
Experimental - in vivo                    51.0
Systematic review and/or meta-analysis    10.0
Clinical-interventional                    8.0
Clinical-interventional                    2.0
Clinical-observational                    31.0
Clinical-observational                    62.0
Systematic review and/or meta-analysis    38.0
dtype: float64

In [441]:
cw_merge.iloc[:,8:].sum()[0]

25.0

In [442]:
#Check to see how many labels per column.
cw_merge.iloc[:,8:].sum()
total = []
for i in range(0,9):
    total.append(cw_merge.iloc[:,8:].sum()[i])
sum(total)

285.0

Total anotations/label prior to .sum() : 154. Will lose a bunch after combining columns together.

In [422]:
#Rename column with suffix so when summed, they are combined together
cw_merge = cw_merge.rename(columns={'Systematic review and/or meta-analysis.1': 'Systematic review and/or meta-analysis'})

In [423]:
list(cw_merge)

['paper_id',
 'sha_x',
 'Study Type',
 'Assignee',
 'cord_uid',
 'sha_y',
 'title',
 'url',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional',
 'Clinical-observational',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

In [443]:
cw_merge

,paper_id,sha_x,Study Type,Assignee,cord_uid,sha_y,title,url,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis
0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],NaN,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,Protein secretion in Lactococcus lactis : an e...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],NaN,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,A Severe Acute Respiratory Syndrome extranet: ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],NaN,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,Pneumothorax and mortality in the mechanically...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],NaN,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,Selection of a set of reliable reference genes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,['questionnaire'],NaN,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,"The Trojan Chicken Study, Minnesota",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,"['experimental', 'clinical']",NaN,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,BCX4430 – A broad-spectrum antiviral adenosine...,https://doi.org/10.1016/j.jiph.2016.04.002,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
256,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,['review'],NaN,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,"The Isolation, Pathogenesis, Diagnosis, Transm...",https://doi.org/10.1016/j.cvex.2010.05.014,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
257,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,['clinical'],NaN,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,Use of a Multiplex RT-PCR Assay for Simultaneo...,https://doi.org/10.1016/s1671-2927(09)60189-9,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
258,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,['clinical'],GD/ lolita,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,Case control study to investigate risk factors...,https://doi.org/10.1016/j.prevetmed.2012.02.008,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [445]:
cw_merge = cw_merge.groupby(cw_merge.columns, axis=1).sum()
cw_merge

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url
0,0,0.0,0.0,0.0,0.0,0.0,['review'],1.0,gdsfkw1b,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,8d7400a2b387820cd391d7df8194642e50402a0c,Protein secretion in Lactococcus lactis : an e...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
1,0,0.0,1.0,0.0,0.0,0.0,['survey'],0.0,e1iaiwc6,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,A Severe Acute Respiratory Syndrome extranet: ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,0,0.0,1.0,0.0,0.0,0.0,['clinical'],0.0,o088rxb5,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,9f63ebfaab049c0968d72166761217b3aaa1fe00,Pneumothorax and mortality in the mechanically...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,0,0.0,0.0,0.0,1.0,0.0,['clinical'],0.0,icp9d998,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,8c640872048570b0a671ec08c83ef6cbebcdd78e,Selection of a set of reliable reference genes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
4,0,0.0,1.0,0.0,0.0,0.0,['questionnaire'],0.0,dh9n3j23,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,c7c4b98517819b787b89bfeee835280593dbf035,"The Trojan Chicken Study, Minnesota",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,0,0.0,0.0,0.0,0.0,1.0,"['experimental', 'clinical']",0.0,j5zlismf,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,703f86c62ceb141bb69656ce5d3ae330d0969976,BCX4430 – A broad-spectrum antiviral adenosine...,https://doi.org/10.1016/j.jiph.2016.04.002
256,0,0.0,1.0,0.0,0.0,1.0,['review'],0.0,es8t6t0j,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,"The Isolation, Pathogenesis, Diagnosis, Transm...",https://doi.org/10.1016/j.cvex.2010.05.014
257,0,0.0,0.0,0.0,0.0,1.0,['clinical'],0.0,91ypig4r,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,8b291c52103b777a67e540fe71f797e0ba58e844,Use of a Multiplex RT-PCR Assay for Simultaneo...,https://doi.org/10.1016/s1671-2927(09)60189-9
258,GD/ lolita,0.0,0.0,0.0,0.0,1.0,['clinical'],0.0,apxv9fd8,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,d205480b1b5704003ff43d9dbb8b531827fcfb7b,Case control study to investigate risk factors...,https://doi.org/10.1016/j.prevetmed.2012.02.008


In [454]:
cw_merge = cw_merge[['Assignee','cord_uid','paper_id','sha_x','sha_y','title','url','Study Type','Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']]

In [460]:
cw_merge.iloc[:,8:].sum()
total = []
for i in range(0,6):
    total.append(cw_merge.iloc[:,8:].sum()[i])
sum(total)

285.0

Great! Numbers were preserved after identical columns were combined together.

In [461]:
cw_merge[cw_merge['Clinical-interventional'] > 1]

,Assignee,cord_uid,paper_id,sha_x,sha_y,title,url,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [462]:
cw_merge[cw_merge['Clinical-observational'] > 1]

,Assignee,cord_uid,paper_id,sha_x,sha_y,title,url,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [463]:
cw_merge[cw_merge['Computational'] > 1]

,Assignee,cord_uid,paper_id,sha_x,sha_y,title,url,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [464]:
cw_merge[cw_merge['Experimental - in vitro'] > 1]

,Assignee,cord_uid,paper_id,sha_x,sha_y,title,url,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [465]:
cw_merge[cw_merge['Experimental - in vivo'] > 1]

,Assignee,cord_uid,paper_id,sha_x,sha_y,title,url,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [466]:
cw_merge[cw_merge['Systematic review and/or meta-analysis'] > 1]

,Assignee,cord_uid,paper_id,sha_x,sha_y,title,url,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
186,0,0cehgc34,0cehgc34,438f183a49a1c6cc580a3edea609bebeb2af0e9f,438f183a49a1c6cc580a3edea609bebeb2af0e9f,Efficacy and gastrointestinal risk of aspirin ...,https://doi.org/10.1016/j.bpg.2012.01.008,"['review', 'clinical']",0.0,0.0,0.0,0.0,0.0,2.0
238,0,hel3h25h,hel3h25h,4f3dfa1d4e3ffba1d199cbdf180baa47805a2e67,4f3dfa1d4e3ffba1d199cbdf180baa47805a2e67,Encephalitis diagnosis using metagenomics: app...,https://doi.org/10.1016/j.jinf.2017.12.014,"['review', 'clinical']",0.0,0.0,0.0,0.0,0.0,2.0


Only column with values greater than 1 is 'Systematic review and/or meta-analysis'. Will change these values into 1.

In [467]:
list(cw_merge)

['Assignee',
 'cord_uid',
 'paper_id',
 'sha_x',
 'sha_y',
 'title',
 'url',
 'Study Type',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

In [468]:
cw_merge = cw_merge.drop(columns = [
                      'Assignee',
                      'cord_uid',
                      'sha_y',
                      'title',
                      'url'])

In [469]:
cw_merge = cw_merge.rename(columns={'sha_x': 'sha'})
cw_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],0.0,0.0,0.0,0.0,0.0,1.0
1,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],0.0,0.0,0.0,0.0,1.0,0.0
2,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],0.0,0.0,0.0,0.0,1.0,0.0
3,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],0.0,1.0,0.0,0.0,0.0,0.0
4,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,['questionnaire'],0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
255,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,"['experimental', 'clinical']",0.0,0.0,1.0,0.0,0.0,0.0
256,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,['review'],0.0,0.0,1.0,0.0,1.0,0.0
257,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,['clinical'],0.0,0.0,1.0,0.0,0.0,0.0
258,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,['clinical'],0.0,0.0,1.0,0.0,0.0,0.0


In [393]:
#Reorder columns
cols = list(kg_merge)
cw_merge = cw_merge[cols]
cw_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],0.0,0.0,0.0,0.0,0.0,1.0
1,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],0.0,0.0,0.0,0.0,0.0,0.0
2,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],0.0,0.0,0.0,0.0,0.0,0.0
3,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],0.0,1.0,0.0,0.0,0.0,0.0
4,dh9n3j23,c7c4b98517819b787b89bfeee835280593dbf035,['questionnaire'],0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
255,j5zlismf,703f86c62ceb141bb69656ce5d3ae330d0969976,"['experimental', 'clinical']",0.0,0.0,1.0,0.0,0.0,0.0
256,es8t6t0j,12aa1a04b599ef61ee16ed00b270af1a9b32fed0,['review'],0.0,0.0,1.0,0.0,0.0,0.0
257,91ypig4r,8b291c52103b777a67e540fe71f797e0ba58e844,['clinical'],0.0,0.0,1.0,0.0,0.0,0.0
258,apxv9fd8,d205480b1b5704003ff43d9dbb8b531827fcfb7b,['clinical'],0.0,0.0,1.0,0.0,0.0,0.0


In [473]:
columns = list(cw_merge.iloc[:,3:])
columns

['Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

In [471]:
#As a reminder
cw_merge[cw_merge['Systematic review and/or meta-analysis'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
186,0cehgc34,438f183a49a1c6cc580a3edea609bebeb2af0e9f,"['review', 'clinical']",0.0,0.0,0.0,0.0,0.0,2.0
238,hel3h25h,4f3dfa1d4e3ffba1d199cbdf180baa47805a2e67,"['review', 'clinical']",0.0,0.0,0.0,0.0,0.0,2.0


In [474]:
count = 0
for i in cw_merge.index:
    for c in columns:
        if (cw_merge[c][count] == 0) | (cw_merge[c][count] == 1.0):
            pass
        elif cw_merge[c][count] > 1.0:
            cw_merge.at[count,c] = 1.0
    count +=1

In [475]:
cw_merge[cw_merge['Systematic review and/or meta-analysis'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


Yes, all values in cw_merge labels are now between 0 and 1.0.

In [476]:
cw_merge.iloc[:,3:].sum()
total2 = []
for i in range(0,6):
    total2.append(cw_merge.iloc[:,3:].sum()[i])
sum(total2) 

283.0

Awesome! This is in line with 2 rows that were greater than 1.0 (2.0) and reduced to 1.0. So loss was a total of 2.0, which is what the output is.

## Now, onto PubMed annotations. Need to:

1. Make sure pm_labels have label per row (ideally 1.0)
2. Merge pm_labels to features
3. Make sure there is still labels per row despite reduction in dataset
4. Merge any same name columns and reconcile values >1

4. (If needed, Change False/True to 0/1)

5. rename label columns to those corresponding to ones in kg_merge/cw_merge
6. Reorder column names

In [485]:
pm_labels = pd.read_csv('metadata_extra_info_from_pubmed_db.csv')
pm_labels

,is_rct,is_systematic_reivew,is_review,is_meta_analysis,is_observational_study,is_case_series,is_clincal_trial,cord_uid,pubmed_id
0,False,False,False,False,False,False,False,zjufx4fo,11742998
1,False,False,False,False,False,False,False,ymceytj3,11980704
2,False,False,False,False,False,False,False,wzj2glte,11447125
3,False,False,False,False,False,False,False,2sfqsfm1,12093723
4,False,False,False,False,False,False,False,i0zym7iq,12456663
...,...,...,...,...,...,...,...,...,...
57361,False,False,False,False,False,False,False,rn4gm3sc,32105468
57362,False,False,True,False,False,False,False,l5oecoot,16931122
57363,False,False,False,False,False,False,False,i4acg2ex,18035462
57364,False,False,True,False,False,False,False,rve0w3dg,8395326


In [489]:
d = {True:1, False:np.nan}
pm_labels = pm_labels.applymap(lambda x: d.get(x,x))
pm_labels

,is_rct,is_systematic_reivew,is_review,is_meta_analysis,is_observational_study,is_case_series,is_clincal_trial,cord_uid,pubmed_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zjufx4fo,11742998.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ymceytj3,11980704.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wzj2glte,11447125.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2sfqsfm1,12093723.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i0zym7iq,12456663.0
...,...,...,...,...,...,...,...,...,...
57361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rn4gm3sc,32105468.0
57362,NaN,NaN,1.0,NaN,NaN,NaN,NaN,l5oecoot,16931122.0
57363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i4acg2ex,18035462.0
57364,NaN,NaN,1.0,NaN,NaN,NaN,NaN,rve0w3dg,8395326.0


In [487]:
list(pm_labels)

['is_rct',
 'is_systematic_reivew',
 'is_review',
 'is_meta_analysis',
 'is_observational_study',
 'is_case_series',
 'is_clincal_trial',
 'cord_uid',
 'pubmed_id']

In [490]:
#Drop all rows that have NaN's across all labels
pm_labels.dropna(subset=['is_rct',
 'is_systematic_reivew',
 'is_review',
 'is_meta_analysis',
 'is_observational_study',
 'is_case_series',
 'is_clincal_trial'], how='all', inplace = True) 
pm_labels

,is_rct,is_systematic_reivew,is_review,is_meta_analysis,is_observational_study,is_case_series,is_clincal_trial,cord_uid,pubmed_id
6,NaN,NaN,1.0,NaN,NaN,NaN,NaN,gi6uaa83,12734001.0
40,NaN,1.0,1.0,NaN,NaN,NaN,NaN,47ema2dq,15518593.0
47,NaN,NaN,NaN,NaN,NaN,NaN,1.0,mtmgur1u,15638943.0
54,NaN,NaN,1.0,NaN,NaN,NaN,NaN,jh9e85c0,15661082.0
76,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0fitbwuv,15833113.0
...,...,...,...,...,...,...,...,...,...
57343,1.0,NaN,NaN,NaN,NaN,NaN,1.0,y8u5jl70,25922328.0
57355,NaN,NaN,1.0,NaN,NaN,NaN,NaN,r35fd6v0,28242560.0
57356,NaN,NaN,1.0,NaN,NaN,NaN,NaN,t2vl0cw6,28377208.0
57362,NaN,NaN,1.0,NaN,NaN,NaN,NaN,l5oecoot,16931122.0


Huge reduction in labels! At least have ~8,000 and not around 1000.

In [491]:
pm_merge = features.merge(pm_labels, how='inner', left_on='paper_id', right_on='cord_uid')
pm_merge

,paper_id,sha,Study Type,is_rct,is_systematic_reivew,is_review,is_meta_analysis,is_observational_study,is_case_series,is_clincal_trial,cord_uid,pubmed_id
0,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']",NaN,NaN,1.0,NaN,NaN,NaN,NaN,gi6uaa83,12734001.0
1,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],NaN,1.0,1.0,NaN,NaN,NaN,NaN,47ema2dq,15518593.0
2,mtmgur1u,18c21db94a22f1358057c6d797164ffe1585b395,['clinical'],NaN,NaN,NaN,NaN,NaN,NaN,1.0,mtmgur1u,15638943.0
3,jh9e85c0,d16e1bd4c6cae1bb55dbc695c5e2ba5105a13921,"['experimental', 'clinical']",NaN,NaN,1.0,NaN,NaN,NaN,NaN,jh9e85c0,15661082.0
4,0fitbwuv,d0c6b0c2d387baae89eb2898969913218b3bedff,['review'],NaN,NaN,1.0,NaN,NaN,NaN,NaN,0fitbwuv,15833113.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5742,swafz389,3941c1d823c466b3c789439259c5030ba8f8aaa3,['clinical'],NaN,NaN,1.0,NaN,NaN,NaN,NaN,swafz389,28062161.0
5743,bupfw5la,e8fb3f9b3a3acda9ebb5cc8ad74624a68c78bbb1,['clinical'],NaN,NaN,1.0,NaN,NaN,NaN,NaN,bupfw5la,31027659.0
5744,1k8v0luz,417cc66866cbe8108fbc116d45ec1d5d2ffbe16e,['clinical'],NaN,NaN,NaN,NaN,1.0,NaN,NaN,1k8v0luz,28734862.0
5745,v6qry3o6,c2d51fee1e73856e4c891437f8349da3eadbe363,['clinical'],NaN,NaN,NaN,NaN,NaN,NaN,1.0,v6qry3o6,22230592.0


Once again, huge reduction. At least, there are >1000 labels.

In [493]:
pm_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5747 entries, 0 to 5746
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   paper_id                5747 non-null   object 
 1   sha                     5216 non-null   object 
 2   Study Type              5747 non-null   object 
 3   is_rct                  163 non-null    float64
 4   is_systematic_reivew    173 non-null    float64
 5   is_review               4996 non-null   float64
 6   is_meta_analysis        100 non-null    float64
 7   is_observational_study  97 non-null     float64
 8   is_case_series          335 non-null    float64
 9   is_clincal_trial        278 non-null    float64
 10  cord_uid                5747 non-null   object 
 11  pubmed_id               5747 non-null   float64
dtypes: float64(8), object(4)
memory usage: 583.7+ KB


Mapping pm_merge labels to cw_merge labels

1. 'is_rct' -> Clinical-interventional
2. 'is_systematic_reivew' -> Systematic review and/or meta-analysis
3. 'is_review' -> Other
4. 'is_meta_analysis' -> Systematic review and/or meta-analysis
5. 'is_observational_study' -> Clinical-observational
6. 'is_case_series' -> Clinical-observational
7. 'is_clincal_trial' -> Clinical-interventional

In [494]:
pm_merge = pm_merge.rename(columns=
{'is_rct': 'Clinical-interventional', 'is_systematic_reivew' : 'Systematic review and/or meta-analysis', 'is_review' : 'Other', 'is_meta_analysis' : 'Systematic review and/or meta-analysis', 'is_observational_study' : 'Clinical-observational', 'is_case_series' : 'Clinical-observational', 'is_clincal_trial' : 'Clinical-interventional'})

In [495]:
pm_merge

,paper_id,sha,Study Type,Clinical-interventional,Systematic review and/or meta-analysis,Other,Systematic review and/or meta-analysis,Clinical-observational,Clinical-observational,Clinical-interventional,cord_uid,pubmed_id
0,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']",NaN,NaN,1.0,NaN,NaN,NaN,NaN,gi6uaa83,12734001.0
1,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],NaN,1.0,1.0,NaN,NaN,NaN,NaN,47ema2dq,15518593.0
2,mtmgur1u,18c21db94a22f1358057c6d797164ffe1585b395,['clinical'],NaN,NaN,NaN,NaN,NaN,NaN,1.0,mtmgur1u,15638943.0
3,jh9e85c0,d16e1bd4c6cae1bb55dbc695c5e2ba5105a13921,"['experimental', 'clinical']",NaN,NaN,1.0,NaN,NaN,NaN,NaN,jh9e85c0,15661082.0
4,0fitbwuv,d0c6b0c2d387baae89eb2898969913218b3bedff,['review'],NaN,NaN,1.0,NaN,NaN,NaN,NaN,0fitbwuv,15833113.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5742,swafz389,3941c1d823c466b3c789439259c5030ba8f8aaa3,['clinical'],NaN,NaN,1.0,NaN,NaN,NaN,NaN,swafz389,28062161.0
5743,bupfw5la,e8fb3f9b3a3acda9ebb5cc8ad74624a68c78bbb1,['clinical'],NaN,NaN,1.0,NaN,NaN,NaN,NaN,bupfw5la,31027659.0
5744,1k8v0luz,417cc66866cbe8108fbc116d45ec1d5d2ffbe16e,['clinical'],NaN,NaN,NaN,NaN,1.0,NaN,NaN,1k8v0luz,28734862.0
5745,v6qry3o6,c2d51fee1e73856e4c891437f8349da3eadbe363,['clinical'],NaN,NaN,NaN,NaN,NaN,NaN,1.0,v6qry3o6,22230592.0


In [282]:
pm_merge = pm_merge.drop(columns = ['cord_uid', 'pubmed_id', 'Other'])

In [284]:
pm_merge = pm_merge.groupby(pm_merge.columns, axis=1).sum()
pm_merge

,Clinical-interventional,Clinical-observational,Study Type,Systematic review and/or meta-analysis,paper_id,sha
0,0,0,['mutagenesis'],0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9
1,0,0,['mutagenesis'],0,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0
2,0,0,"['genetic', 'histories']",0,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac
3,0,0,['genetic'],0,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48
4,0,0,['genetic'],0,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774
...,...,...,...,...,...,...
21238,0,0,['clinical'],0,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9
21239,0,0,['review'],0,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04
21240,0,0,['review'],0,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6
21241,0,0,['bioinformatics'],0,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de


In [286]:
#Make sure all values in pm_merge label columns are 0/1
pm_merge[pm_merge['Clinical-interventional'] > 1]

,Clinical-interventional,Clinical-observational,Study Type,Systematic review and/or meta-analysis,paper_id,sha
209,2,0,['clinical'],0,543aq9dx,d94115064a8168415d634f654869a4009e4c3c70; 3d29...
296,2,0,['clinical'],0,iu1d9i57,8a244626a996d30b62a52bb8705b889e74f3a5cf
403,2,0,['clinical'],0,2egeyh0j,87a937ec2cd698b7d24ade1803a22852747ec8c7
559,2,0,['clinical'],0,19sejitq,073d9b195e4d3e325a8ec7cd30b9ec23bb0a00a7; 21f0...
615,2,0,['clinical'],0,f5ka7eru,cdb117ad4ac490f1e48e3738d5ccdaafe8916086
...,...,...,...,...,...,...
20921,2,0,['clinical'],0,qjfq1xuj,ae9e3663f5d54898189a6b1de0141edd12a955a5
20933,2,0,['clinical'],0,hduefdt3,59932fb1f0b98d054deab83dbf18caa8396cb6e8; 8c1b...
21015,2,0,"['questionnaire', 'clinical']",0,jsfrktix,9630b95696daa16ef7377ab2fb1373b4cdf86058
21025,2,0,['clinical'],0,dm3u7syh,a652d618a8a622b7b2fa20d6e0eb2ffe7a3657d1


In [287]:
pm_merge[pm_merge['Clinical-observational'] > 1]

,Clinical-interventional,Clinical-observational,Study Type,Systematic review and/or meta-analysis,paper_id,sha
18383,0,2,"['review', 'clinical']",0,6tq0kcfd,8186586de026807b862de876abfeef625a9a78cb


In [288]:
pm_merge[pm_merge['Systematic review and/or meta-analysis'] > 1]

,Clinical-interventional,Clinical-observational,Study Type,Systematic review and/or meta-analysis,paper_id,sha
337,0,0,"['review', 'meta-analysis', 'pooled analysis']",2,67bhdlzy,0
1027,0,0,"['review', 'retrospective cohort', 'clinical']",2,ox4dwecj,04d9df812fd77e1231263d42ab45e570cd94d956
1439,0,0,"['review', 'meta-analysis']",2,giw4z3ei,142a615ffb970d12beaa9597bff2b9c49da4bb96
1440,0,0,['review'],2,8gfujgxq,428d1091cf63872ea81cb3c1632d76c4813748a1; 8244...
1902,0,0,"['review', 'meta-analysis', 'clinical']",2,vuzbswvq,9e5c4cfb4582c7aa5e60455ae948554981054c47
...,...,...,...,...,...,...
16870,0,0,['review'],2,htd7dmn9,9a12c1852bf079e75fe2b1bb3186e468081a164e
17149,0,0,"['review', 'meta-analysis']",2,da1xab1s,3f1a42a65b20b1ba6e211e411f459cd8ea1af14b
18223,0,0,"['review', 'meta-analysis']",2,b5tduubu,6c8ba1999a10405500b412d0b644f0c47d066ec2
19245,0,0,"['review', 'clinical']",2,f5mxb040,c04544e941bb99c37c79f23baaf4cf9e64bdb386


In [291]:
#add blank columns to equal kg_merge/cw_merge columns
c = {'Computational': 0, 'Experimental - in vitro': 0, 'Experimental - in vivo': 0}
pm_merge = pm_merge.assign(**c)
pm_merge

,Clinical-interventional,Clinical-observational,Study Type,Systematic review and/or meta-analysis,paper_id,sha,Computational,Experimental - in vitro,Experimental - in vivo
0,0,0,['mutagenesis'],0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,0,0,0
1,0,0,['mutagenesis'],0,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,0,0,0
2,0,0,"['genetic', 'histories']",0,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,0,0,0
3,0,0,['genetic'],0,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,0,0,0
4,0,0,['genetic'],0,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,0,0,0
...,...,...,...,...,...,...,...,...,...
21238,0,0,['clinical'],0,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,0,0,0
21239,0,0,['review'],0,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,0,0,0
21240,0,0,['review'],0,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,0,0,0
21241,0,0,['bioinformatics'],0,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,0,0,0


In [292]:
#Reorder columns
cols = list(cw_merge)
pm_merge = pm_merge[cols]
pm_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis'],0,0,0,0,0,0
1,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis'],0,0,0,0,0,0
2,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']",0,0,0,0,0,0
3,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic'],0,0,0,0,0,0
4,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic'],0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
21238,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical'],0,0,0,0,0,0
21239,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review'],0,0,0,0,0,0
21240,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review'],0,0,0,0,0,0
21241,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics'],0,0,0,0,0,0


In [294]:
columns = list(pm_merge.iloc[:,3:])
columns

['Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

In [295]:
count = 0
for i in pm_merge.index:
    for c in columns:
        if (pm_merge[c][count] == 0) | (pm_merge[c][count] == 1.0):
            pass
        elif pm_merge[c][count] > 1:
            pm_merge.at[count,c] = 1
    count +=1

In [296]:
#Check that all values in label columns are 0/1
pm_merge[pm_merge['Clinical-interventional'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [297]:
pm_merge[pm_merge['Clinical-observational'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


In [298]:
pm_merge[pm_merge['Systematic review and/or meta-analysis'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


Great! All values in label columns are either <= 1.

Now merge all theree tables and drop duplicates.

In [302]:
pdList = [kg_merge, cw_merge, pm_merge] 
StudyTypeAnnotations =  pd.concat(pdList).drop_duplicates()
StudyTypeAnnotations

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],0.0,0.0,0.0,0.0,0.0,1.0
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",0.0,0.0,0.0,0.0,1.0,0.0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",0.0,0.0,0.0,0.0,1.0,0.0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],0.0,0.0,0.0,0.0,1.0,0.0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
21237,cf2axvzq,7d4fc3715d86426fd7c0a3e998e27a8c6e6330b5,['questionnaire'],0.0,0.0,0.0,0.0,0.0,0.0
21238,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical'],0.0,0.0,0.0,0.0,0.0,0.0
21239,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review'],0.0,0.0,0.0,0.0,0.0,0.0
21241,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics'],0.0,0.0,0.0,0.0,0.0,0.0


In [301]:
#kg_merge + cw_merge + pm_merge DFs' rows summation
398+427+21243

22068

In [306]:
StudyTypeAnnotations.iloc[:,3:].sum()

Computational                              62.0
Experimental - in vitro                    58.0
Experimental - in vivo                     51.0
Clinical-interventional                   304.0
Clinical-observational                    679.0
Systematic review and/or meta-analysis    296.0
dtype: float64

In [310]:
total = []
for i in range(0,6):
    total.append(StudyTypeAnnotations.iloc[:,3:].sum()[i])
sum(total)    

1450.0

In [312]:
#Need to check if all DF's to see if there are at least 1 annotation per publication. If not, need to make sure they are filled/deleted.

In [ ]:
#StudyTypeAnnotations.to_csv("StudyTypeAnnotations.csv", index=False)